In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.preprocessing import OrdinalEncoder

In [3]:
cd ..

C:\Projects\python\recommender


In [4]:
from utils import build_logger

logger = build_logger()

## Read user dataframe and filter users and items

In [5]:
user_min = 5
item_min = 5

df = pd.read_csv("inputs/ml-100k/u.data",
                 header=None,
                 sep="\t",
                 names=["user_id", "item_id", "rating", "time"],
                 dtype={
                     'user_id': 'int32',
                     'item_id': 'int32',
                     'rating': 'int32',
                     'time': 'int32'
                 })
df.head()

,user_id,item_id,rating,time
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [6]:
user_counts = df["user_id"].value_counts()
user_counts.head()

405    737
655    685
13     636
450    540
276    518
Name: user_id, dtype: int64

In [7]:
item_counts = df["item_id"].value_counts()
print('previous item shape: {}'.format(item_counts.size))

previous item shape: 1682


In [8]:
# get user and tiem category info
user_counts = user_counts[user_counts >= user_min]
item_counts = item_counts[item_counts >= item_min]

In [9]:
print("next user size: {}".format(user_counts.size))
print('next item size: {}'.format(item_counts.size))

next user size: 943
next item size: 1349


In [10]:
df = df[df.user_id.isin(user_counts.index)]
df = df[df.item_id.isin(item_counts.index)]

In [11]:
print('Second pass')
print('num_users = {}'.format(df["user_id"].unique().size))
print('num_items = {}'.format(df["item_id"].unique().size))
print('df_shape  = {}'.format(df.shape))

Second pass
num_users = 943
num_items = 1349
df_shape  = (99287, 4)


In [12]:
# Build user and item matrix
user_one_hot = sp.identity(user_counts.size).tocsr()
item_one_hot = sp.identity(item_counts.size).tocsr()

In [13]:
user_one_hot

<943x943 sparse matrix of type '<class 'numpy.float64'>'
	with 943 stored elements in Compressed Sparse Row format>

In [14]:
item_one_hot

<1349x1349 sparse matrix of type '<class 'numpy.float64'>'
	with 1349 stored elements in Compressed Sparse Row format>

## Read user dataframe

In [15]:
base_cols = ['movie_id', 'movie_title', 'release_date', 'video_release_date','imdb_url']
movie_cat_cols = ['unkown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy',
                            'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
                            'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

In [16]:
item_df = pd.read_csv('./inputs/ml-100k/u.item',
                      header=None,
                      index_col=False,
                      sep='|')
item_df.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [17]:
item_df.columns = base_cols + movie_cat_cols
item_df.head()

,movie_id,movie_title,release_date,video_release_date,imdb_url,unkown,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [18]:
item_df = item_df[['movie_id'] + movie_cat_cols]
item_df.head()

,movie_id,unkown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


### Join data and users information

In [20]:
result_df = pd.merge(left=df, right=item_df, left_on=['item_id'], right_on=['movie_id'], how='left')
result_df.head()

,user_id,item_id,rating,time,movie_id,unkown,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,196,242,3,881250949,242,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,186,302,3,891717742,302,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
2,22,377,1,878887116,377,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,244,51,2,880606923,51,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,1
4,166,346,1,886397596,346,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
df = df.sort_values(by=['user_id', 'time'])
df = df.reset_index(drop=True)
df.head()

,user_id,item_id,rating,time
0,1,168,5,874965478
1,1,172,5,874965478
2,1,165,5,874965518
3,1,156,4,874965556
4,1,196,5,874965677


In [14]:
# Add previous item
df['prev_item_id'] = df.item_id
df.prev_item_id = df.prev_item_id.shift(periods=1).fillna(0).astype(np.int32)
df.head()

,user_id,item_id,rating,time,prev_item_id
0,1,168,5,874965478,0
1,1,172,5,874965478,168
2,1,165,5,874965518,172
3,1,156,4,874965556,165
4,1,196,5,874965677,156


In [15]:
# Negative sampling
df['neg_item_id'] = df.item_id.sample(df.shape[0]).values
df.head()

,user_id,item_id,rating,time,prev_item_id,neg_item_id
0,1,168,5,874965478,0,8
1,1,172,5,874965478,168,282
2,1,165,5,874965518,172,1247
3,1,156,4,874965556,165,977
4,1,196,5,874965677,156,288


In [16]:
df.dtypes

user_id         int32
item_id         int32
rating          int32
time            int32
prev_item_id    int32
neg_item_id     int32
dtype: object

In [17]:
# split train and test ddataframe
df = df.sort_values(by=['time'])
duplicate_mask = df.duplicated(subset=['user_id'], keep='last')
remain_df = df[duplicate_mask]
test_df = df[~duplicate_mask]
duplicate_mask = remain_df.duplicated(subset=['user_id'], keep='last')
train_df = remain_df[duplicate_mask]
valid_df = remain_df[~duplicate_mask]

In [18]:
logger.info("train shape {}".format(train_df.shape))
logger.info("valid shape {}".format(valid_df.shape))
logger.info("test shape {}".format(test_df.shape))

2019-09-02 21:17:42,708 - C:\Projects\python\recommender\utils.py - INFO - train shape (97401, 6)
2019-09-02 21:17:42,709 - C:\Projects\python\recommender\utils.py - INFO - valid shape (943, 6)
2019-09-02 21:17:42,709 - C:\Projects\python\recommender\utils.py - INFO - test shape (943, 6)


In [19]:
# encode feature
cat_names = ['user_id', 'item_id', 'prev_item_id']
ordinal_encoder = OrdinalEncoder(categories='auto', dtype='int32')

data = ordinal_encoder.fit_transform(train_df[cat_names])
train_df[cat_names] = data

data = ordinal_encoder.transform(valid_df[cat_names])
valid_df[cat_names] = data

data = ordinal_encoder.transform(test_df[cat_names])
test_df[cat_names] = data

c:\projects\python\recommender\.venv\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
c:\projects\python\recommender\.venv\lib\site-packages\pandas\core\frame.py:3395: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc._setitem_with_indexer((slice(None), indexer), value)
c:\projects\python\recommender\.venv\lib\site-packages\pandas\core\frame.py:3367: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-ve

In [20]:
# Set first item non for each user
train_df.sort_values(by=['user_id'])
first_mask = ~train_df.duplicated(subset=['user_id'], keep='first')
train_df['prev_item_id'][first_mask] = -1

c:\projects\python\recommender\.venv\lib\site-packages\pandas\core\generic.py:8682: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
c:\projects\python\recommender\.venv\lib\site-packages\IPython\core\interactiveshell.py:3296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [21]:
train_df.head()

,user_id,item_id,rating,time,prev_item_id,neg_item_id
25222,258,253,4,874724710,-1,64
25223,258,284,4,874724727,254,100
25224,258,296,4,874724754,285,702
25225,258,183,4,874724781,297,1473
25226,258,171,4,874724843,184,1142


In [22]:
train_df.dtypes

user_id         int32
item_id         int32
rating          int32
time            int32
prev_item_id    int32
neg_item_id     int32
dtype: object

In [23]:
train_df = train_df[cat_names]
valid_df = valid_df[cat_names]
test_df = test_df[cat_names]

In [24]:
one_data = train_df.iloc[1]
one_data

user_id         258
item_id         284
prev_item_id    254
Name: 25223, dtype: int32

In [22]:
one_data.values

array([258, 284, 253])

In [34]:
user_id = train_df.iloc[1]['user_id']
prev_item_id = train_df.iloc[1]['prev_item_id']
item_id = train_df.iloc[1]['item_id']

In [35]:
user = user_one_hot[user_id]
prev_item = item_one_hot[prev_item_id]
item = item_one_hot[item_id]

In [36]:
feats = sp.hstack([user, prev_item, item])
feats

<1x3641 sparse matrix of type '<class 'numpy.float64'>'
	with 3 stored elements in COOrdinate format>

In [38]:
feats.nonzero()

(array([0, 0, 0], dtype=int32), array([ 258, 1197, 2576], dtype=int32))

## Build pytorch Dataset

In [24]:
from torch.utils.data import Dataset, DataLoader

class MovieLenDataset(Dataset):
    def __init__(self, df: pd.DataFrame, dataset_type: str):
        self.dataset_type = dataset_type
        self.df = df

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idx):
        return self.df.iloc[idx].values

In [26]:
train_ds = MovieLenDataset(train_df, 'train')
valid_ds = MovieLenDataset(valid_df, 'valid')
test_ds = MovieLenDataset(test_df, 'test')

In [28]:
print(train_ds[1])
print(type(train_ds[1]))

[258 284 253]
<class 'numpy.ndarray'>


In [31]:
data_bunch = DataLoader(train_ds, batch_size=4, shuffle=True)

In [32]:
for i_batch, sample_batch in enumerate(data_bunch):
    print("num of batch", i_batch)
    print("sample batch", sample_batch)
    
    break

num of batch 0
sample batch tensor([[  37,   66, 1000],
        [ 120,   11,   49],
        [ 590,  170,  463],
        [ 234,   78,  235]], dtype=torch.int32)


In [40]:
import torch as T
from torch import Tensor

def cat_collate(batch) -> Tensor:
    return T.tensor(batch, dtype=T.long)

In [41]:
data_bunch = DataLoader(train_ds, batch_size=4,collate_fn=cat_collate, shuffle=True)

In [42]:
for i_batch, sample_batch in enumerate(data_bunch):
    print("num of batch", i_batch)
    print("sample batch", sample_batch)
    
    break

num of batch 0
sample batch tensor([[245, 467, 670],
        [442, 243, 341],
        [832, 394, 379],
        [647, 838, 668]])


In [43]:
sample_batch.dtype

torch.int64

In [44]:
sample_batch.shape

torch.Size([4, 3])

In [47]:
{
    name: cat_array
    for name, cat_array in zip(cat_names, ordinal_encoder.categories_)
}

{'user_id': array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
         27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
         40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
         53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
         66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
         79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
         92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
        105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
        118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
        131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
        144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
        157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
        170, 171, 172, 173, 174, 175, 17